# Lesson 3: Enable Logging

### Import all needed packages

In [8]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [9]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [10]:
log_group_name = '/my/amazon/bedrock/logs'

In [11]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' already exists.


In [12]:
# This is for the large log data that has to be stored from cloudwatch to s3
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
#this is to directly store the logs into s3
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [13]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': '12471319-6606-4e8b-8852-854f3af07266',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Jun 2024 06:02:51 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '12471319-6606-4e8b-8852-854f3af07266'},
  'RetryAttempts': 0}}

In [14]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': 'c9ecc860-d559-4084-8aa2-2a8bf4e3f34e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Jun 2024 06:02:56 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c9ecc860-d559-4084-8aa2-2a8bf4e3f34e'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::144537045289:role/c99355a2566044l6914547t1w14453704528-LoggingIAMRole-AW3iZ4VpkjTY',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l6914547t1w144537045-loggings3bucket-i81a5j4abwd6',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l6914547t1w144537045-loggings3bucket-i81a5j4abwd6',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [15]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [16]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet in the Star Wars universe, first mentioned in the 1977 film Star Wars: Episode IV - A New Hope. The planet is home to a species of sentient, reptilian creatures known as the Jawas. The Jawas are known for their scavenging and trading practices, and they have a reputation for being cunning and treacherous.

The planet Foobar is located in the Outer Rim Territories, a region of space beyond the reaches of the Galactic Republic. It is a rocky, desert-like world with few natural resources and harsh environmental conditions. The Jawas are native to Foobar and have adapted to survive in these conditions, with their long, flexible necks and sharp claws allowing them to scavenge for food and tools.

The first mention of Foobar comes in the opening scene of Star Wars: Episode IV - A New Hope, when the spaceship Millennium Falcon is chased by the Imperial Star Destroyer Executor. The Millennium Falcon is forced to crash-land on Foobar, where the crew is captured by 

In [17]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-06-14T06:03:39Z",
    "accountId": "144537045289",
    "identity": {
        "arn": "arn:aws:sts::144537045289:assumed-role/voclabs/user3329832=0"
    },
    "region": "us-west-2",
    "requestId": "fac02a11-ac43-4773-afbe-0731e4f512fb",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "inputTextTokenCount": 10,
    

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [18]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [19]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
